<a href="https://colab.research.google.com/github/PaulYYLin/MINST_KaggleComp/blob/main/MINST_kaggle_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Kaggle Connection

In [ ]:
!pip install kaggle
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json



Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 65 bytes


In [ ]:
!kaggle competitions list

In [ ]:
!kaggle competitions download "digit-recognizer"

 91% 14.0M/15.3M [00:00<00:00, 70.8MB/s]
100% 15.3M/15.3M [00:00<00:00, 71.3MB/s]


In [ ]:
!unzip digit-recognizer.zip

Archive:  digit-recognizer.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


### Data Description
Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('train.csv')

The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

## Data Preparation

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
# Data split
x = train.iloc[:int(len(train)*0.8),1:]
y = train.iloc[:int(len(train)*0.8),0]

x = tf.reshape(x.values,[-1,28,28,1])

x_val = train.iloc[int(len(train)*0.8):,1:]
y_val = train.iloc[int(len(train)*0.8):,0]

x_val = tf.reshape(x_val.values,[-1,28,28,1])

In [ ]:
# Data Normalize & Balance
def feature_scale(x,y):
  x = tf.cast(x, dtype = tf.float32)/255
  y = tf.cast(y, dtype = tf.int32)
  return x,y

data = tf.data.Dataset.from_tensor_slices((x,y))
data = data.map(feature_scale).shuffle(10000).batch(128)

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.map(feature_scale).batch(128)

print((x.shape, y.shape),(x_val.shape, y_val.shape))


(TensorShape([33600, 28, 28, 1]), (33600,)) (TensorShape([8400, 28, 28, 1]), (8400,))


## Model Compile

#### DNN

In [ ]:
#定義Model

model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(16, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.build(input_shape=[1,28*28])

In [ ]:
tf.keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

tf.keras.losses https://www.tensorflow.org/api_docs/python/tf/keras/losses

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics = ['crossentropy'],
)
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_54 (Dense)            (1, 256)                  200960    
                                                                 
 dense_55 (Dense)            (1, 128)                  32896     
                                                                 
 dense_56 (Dense)            (1, 64)                   8256      
                                                                 
 dense_57 (Dense)            (1, 32)                   2080      
                                                                 
 dense_58 (Dense)            (1, 16)                   528       
                                                                 
 dense_59 (Dense)            (1, 10)                   170       
                                                                 
Total params: 244,890
Trainable params: 244,890
Non-tr

#### CNN

##### VGG10 Method
- 10 hidden layer + 3 fully connected layer


In [ ]:
# Convolution & Pooling
VGG_layers_10 = [

 #stack1
    tf.keras.layers.Conv2D(28,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.Conv2D(28,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    #stack2
    tf.keras.layers.Conv2D(56,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.Conv2D(56,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='same'),
    #stack3
    tf.keras.layers.Conv2D(112,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.Conv2D(112,kernel_size=[3,3],padding='same',activation=tf.nn.relu),
    tf.keras.layers.Conv2D(112,kernel_size=[1,1],padding='same',activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=[2,2],strides=5,padding='same'),
    tf.keras.layers.AveragePooling2D(pool_size=[2,2],strides=2,padding='same'),

]

# FC & Flat
fc_layers=[
tf.keras.layers.Dense(56,activation=tf.nn.relu),
tf.keras.layers.Dense(28,activation=tf.nn.relu),
tf.keras.layers.Dense(10,activation=None)
]

In [ ]:
vgg_10 = tf.keras.Sequential(VGG_layers_10)
vgg_10.build(input_shape=[None,28,28,1]) # B&W image >> 1 channel

fc = tf.keras.Sequential(fc_layers)
fc.build(input_shape=[None,112])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [ ]:

vgg_10.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 28, 28, 28)        280       
                                                                 
 conv2d_8 (Conv2D)           (None, 28, 28, 28)        7084      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 28)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 56)        14168     
                                                                 
 conv2d_10 (Conv2D)          (None, 14, 14, 56)        28280     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 7, 7, 56)          0         
 g2D)                                                 

In [ ]:
fc.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 56)                6328      
                                                                 
 dense_1 (Dense)             (None, 28)                1596      
                                                                 
 dense_2 (Dense)             (None, 10)                290       
                                                                 
Total params: 8214 (32.09 KB)
Trainable params: 8214 (32.09 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Model Fit / Evaluate

In [ ]:
# history = fc.fit(tf.reshape(vgg.(x), [-1,256]), y, epochs=10, validation_split = 0.1)

EPOCHS = 5
variables = vgg_10.variables + fc.variables

for i in range(EPOCHS):
  for step,(x_t,y_t) in enumerate(data):
    with tf.GradientTape() as tape:
      logits = vgg_10(x_t)
      logits = tf.reshape(logits,[-1,112])
      logits = fc(logits)
      y_one_hot = tf.one_hot(y_t,depth=10)
      loss = tf.losses.categorical_crossentropy(y_one_hot,logits, from_logits=True)
      loss = tf.reduce_mean(loss)
    grads = tape.gradient(loss,variables)
    optimizer.apply_gradients(zip(grads,variables))

    if step%100 ==0:
      print('Epoch: ',i, '\tStep: ',step, '\tloss: ',float(loss))


  # Validation by currect model
  total_correct  = 0
  total_num = 0

  for x_v,y_v in val_data:

    logits = vgg_10(x_v)
    logits = tf.reshape(logits, [-1,112])

    logits = fc(logits)

    prob = tf.nn.softmax(logits, axis=1)
    pred = tf.argmax(prob, axis=1)

    pred = tf.cast(pred,dtype=tf.int32)
    correct = tf.equal(pred, y_v)

    result = tf.reduce_sum(tf.cast(correct, dtype=tf.int32))

    total_correct += int(result)
    total_num += x_v.shape[0]

  acc = total_correct /total_num
  print(i, '\tAcc: ',acc, total_correct, total_num)



Epoch:  0 	Step:  0 	loss:  0.045970965176820755
Epoch:  0 	Step:  100 	loss:  0.046672701835632324
Epoch:  0 	Step:  200 	loss:  0.014219509437680244
0 	Acc:  0.9767857142857143 8205 8400
Epoch:  1 	Step:  0 	loss:  0.05639557167887688
Epoch:  1 	Step:  100 	loss:  0.07590631395578384
Epoch:  1 	Step:  200 	loss:  0.09303268790245056
1 	Acc:  0.9778571428571429 8214 8400
Epoch:  2 	Step:  0 	loss:  0.04475398361682892
Epoch:  2 	Step:  100 	loss:  0.016465773805975914
Epoch:  2 	Step:  200 	loss:  0.05451316758990288
2 	Acc:  0.9795238095238096 8228 8400
Epoch:  3 	Step:  0 	loss:  0.017072685062885284
Epoch:  3 	Step:  100 	loss:  0.026743147522211075
Epoch:  3 	Step:  200 	loss:  0.007277385797351599
3 	Acc:  0.9795238095238096 8228 8400
Epoch:  4 	Step:  0 	loss:  0.04788359999656677
Epoch:  4 	Step:  100 	loss:  0.010590435937047005
Epoch:  4 	Step:  200 	loss:  0.025799443945288658
4 	Acc:  0.9796428571428571 8229 8400


In [ ]:
vgg_10.save('VGG_Ep10_test2')

## Model Predict

In [ ]:
new_model = tf.keras.models.load_model('VGG_Ep10_test2')

In [ ]:
test = pd.read_csv('test.csv')
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_x = tf.reshape(test.values,[-1,28,28,1])
test_x = tf.cast(test_x, dtype = tf.float32)/255

test_data = tf.data.Dataset.from_tensor_slices(test_x)
test_data = test_data.batch(280)

final_hat = []

for ele in test_data:
  logits = vgg_10(ele)
  logits = tf.reshape(logits, [-1,112])

  logits = fc(logits)
  prob = tf.nn.softmax(logits, axis=1)
  pred = tf.argmax(prob, axis=1)
  pred = tf.cast(pred,dtype=tf.int32)

  for n in pred:
    final_hat.append(n.numpy())


In [ ]:
result = pd.read_csv('sample_submission.csv')
result['Label'] = final_hat
result.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [ ]:
result.to_csv('vgg_hat.csv', index=False)